In [1]:
from roma_dspy import RecursiveSolverModule, RecursiveSolver, Executor, Atomizer, Planner, Aggregator
import dspy 

executor_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)
atomizer_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)
planner_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)
aggregator_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)

# Initialize modules
atomizer = Atomizer(lm=atomizer_lm)
planner = Planner(lm=planner_lm)
executor = Executor(lm=executor_lm)
aggregator = Aggregator(lm=aggregator_lm)

# Create solver
solver = RecursiveSolver(
    atomizer,
    planner,
    executor,
    aggregator,
    max_depth=1
)

dspy_module = RecursiveSolverModule(solver=solver)

In [2]:
task_goal = "Write me a blog post about the benefits of using DSPy."

root = await dspy_module.aforward(task_goal)

2025/09/30 20:25:19 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


In [ ]:
from roma_dspy.visualizer import TreeVisualizer, ContextFlowVisualizer, LLMTraceVisualizer

# Show the full execution report
tree = TreeVisualizer(use_colors=True, show_timing=True)
print(tree.visualize(root.completed_task, dag=dspy_module._solver.last_dag))

In [ ]:
context_viz = ContextFlowVisualizer(use_colors=True)
print(context_viz.visualize(dspy_module, show_full_context=True))


In [ ]:
viz = LLMTraceVisualizer(show_metrics=False, show_summary=False, verbose=True)
trace = viz.visualize(dspy_module)

In [ ]:
# Debug: Check what modules were recorded for the root task
root_task = dspy_module._solver.last_dag.get_node(list(dspy_module._solver.last_dag.graph.nodes())[0])
print("Root task execution_history keys:", list(root_task.execution_history.keys()))
for module_name, result in root_task.execution_history.items():
    print(f"\n{module_name}:")
    print(f"  Duration: {result.duration:.2f}s")
    print(f"  Input type: {type(result.input)}")
    print(f"  Output type: {type(result.output)}")

In [3]:
dspy_module._solver.last_dag.get_execution_id()

'e4b08850-9c6e-48c6-9ee6-1420abbddac6'